In [201]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime, date
sns.set()
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [118]:
df1 = pd.read_excel("Support tickets zoho.xlsx",sheet_name='All open tickets')
df1.head()

,Classifications,Category (Activity),Account Name,Ticket Id,CS Owner,Priority (Ticket),Created By (Ticket),Created Time (Ticket),Due Date,Ticket Age,...,Type 2,RCA Category,STAGE,Team,KB shared,Crawling,Happiness Rating,Support Feedback,Ticket Description,Resolution
0,-,-,-,5018.0,-,-,Support Admin,08/22/2022 07:24 PM,-,0.615243,...,-,-,-,-,-,False,-,-,NaN,-
1,-,-,Bluebeam,5020.0,Prashant Begra,Low,Support Admin,08/22/2022 09:21 PM,02/18/2023 10:21 PM,0.534479,...,-,-,-,-,-,False,-,-,Content Source: https://bl091912p.searchunify....,-
2,-,-,Broadcom,4967.0,Divanshi Arora,Low,Support Admin,08/10/2022 08:27 PM,02/06/2023 09:27 PM,12.571470,...,-,-,-,-,-,False,-,-,Content Source: BSN External Search Search Cli...,-
3,-,-,Broadcom,5013.0,Divanshi Arora,Low,Support Admin,08/19/2022 09:06 PM,02/15/2023 10:06 PM,3.544664,...,-,-,-,-,-,False,-,-,Content Source: https://bc102102s.searchunify....,-
4,-,-,Cohesity,5011.0,Deepak Upadhyay,Low,Support Admin,08/19/2022 01:44 PM,02/15/2023 02:44 PM,3.851296,...,-,-,-,-,-,False,-,-,Content Source: support.cohesity.com Search Cl...,-


In [119]:
df1.dropna(subset=['Ticket Description'], inplace=True)

In [120]:
df1.to_csv('Support tickets zoho - All open tickets.csv',index=False)

In [225]:
len(df1)

50

In [121]:
df2 = pd.read_excel("Support tickets zoho.xlsx",sheet_name='close tickets last 6 months')
len(df2)

432

In [122]:
df2.dropna(subset=['Ticket Description'], inplace=True)

In [123]:
df2.head()

,Status (Ticket),Ticket Id,Account Name,Created Time (Ticket),Ticket Closed Time,Subject,Agent Responded Time,Category (Ticket),Classifications,Due Date,First Response Time in Business Hours,Ticket Owner,Account Owner,Support Owner,RCA Category,Resolution,Ticket Description
0,Closed,2338.0,Kronos Incorporated,10/22/2020 06:46 PM,07/07/2022 05:54 PM,Search not returning expected results.,03/30/2021 07:20 PM,Search Client,Configurations,04/20/2021 06:46 PM,3816:33:00,Support Team,Neeraj Trikha,Omkar Dhiman,Backend Configurations,Client closed this case at their end. This was...,Search internally in production seems to not r...
1,Closed,2927.0,Cohesity,06/08/2021 03:34 PM,06/07/2022 07:33 PM,The Agent Console in SalesForce takes a lot of...,05/25/2022 11:04 AM,Developers,Enhancement,07/20/2021 07:34 AM,3882:32:00,Support Team,Deepak Upadhyay,Omkar Dhiman,Major Enhancement,we have an improved and efficient version of c...,"To expedite your request, please provide the f..."
2,Closed,3320.0,Powerschool,08/05/2021 01:04 AM,04/06/2022 11:13 PM,Not All Products are Showing When All Content ...,-,Search Client,Enhancement,02/01/2022 02:04 AM,-,Support Team,Lynette Ledoux,Sumit Sindhu,Major Enhancement,Query: I see most products listed when I selec...,Content Source: Alll Search Client: PowerSchoo...
3,Closed,3322.0,Powerschool,08/05/2021 02:00 AM,04/06/2022 11:14 PM,Product Help Space - Mapping,-,Content Source,Enhancement,02/01/2022 03:00 AM,-,Support Team,Lynette Ledoux,Sumit Sindhu,Major Enhancement,Query: Mapping is not aligned properly with Pr...,Content Source: Product Help Search Client: Po...
4,Closed,3602.0,Geotab,10/09/2021 03:15 AM,08/19/2022 02:11 AM,Unable to edit an article,10/20/2021 09:09 PM,Addons,Enhancement,11/19/2021 07:15 PM,280:08:00,Support Team,Neeraj Trikha,Sumit Sindhu,Custom Code fix,Query: Unable to edit an article\nSU team: Fun...,Content Source: Community Search Client: Geota...


In [131]:
df2.Classifications.value_counts()

Configurations    145
Incident           80
Enhancement        76
Question           59
Bug                55
Others             12
Change Request      2
-                   2
Name: Classifications, dtype: int64

In [132]:
df2.to_csv('Support tickets zoho - close tickets last 6 months.csv',index=False)

In [161]:
#model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

## TRAIN SET

In [162]:
df_train = df2['Ticket Description'].tolist()
df_train_unique= list(set(df_train))

In [163]:
len(df_train_unique)

430

# TEST SET

In [164]:
df_test = df1['Ticket Description'].tolist()
df_test_unique = list(set(df_test))

In [165]:
len(df_test_unique)

50

In [166]:
train_sentence_embedding = model.encode(df_train_unique)

In [167]:
test_sentence_embedding = model.encode(df_test_unique)

In [168]:
similarity_score = cosine_similarity([test_sentence_embedding[1]],train_sentence_embedding[0:],).flatten()

In [169]:
pd.DataFrame({"Test_data":df_test_unique[1],"Sentences":df_train_unique[0:],"Similarity_score":similarity_score})

,Test_data,Sentences,Similarity_score
0,.,Content Source: Rehab Search Client: Rehab Pro...,-0.034684
1,.,"Hi Sahil, Can you share with us the field deta...",0.020759
2,.,Content Source: LightningSB (ctoFULL-5/14) Sea...,-0.032738
3,.,Content Source: SalesForce Search Client: ? Pr...,0.010836
4,.,Content Source: Salesforce Knowledge Search Cl...,-0.045453
...,...,...,...
425,.,Content Source: Revenera Community Search Clie...,-0.000646
426,.,Content Source: community Search Client: commu...,-0.071222
427,.,Content Source: Higher logic community Search ...,-0.091787
428,.,Content Source: N/A Search Client: All Product...,0.090832


In [170]:
train_embedding = model.encode(df_train_unique)
def simalarity(test_set):
    test_embedding = model.encode(df_test_unique)
    print(len(test_embedding))
    append_data = []
    for query in range(0,len(test_embedding)):
        similarity_score = cosine_similarity([test_embedding[query]],train_embedding[0:],).flatten()
        data = pd.DataFrame({"Test_set_query":df_test_unique[query],"Search_query":df_train_unique,"similarity_Score":similarity_score})
        append_data.append(data)
    appended_data = pd.concat(append_data)
    return appended_data

In [171]:
cosine_data = simalarity(df_test)

50


In [172]:
cosine_data = cosine_data.sort_values(['similarity_Score'], ascending=[False])

In [173]:
cosine_data.head(50)

,Test_set_query,Search_query,similarity_Score
173,Content Source: All Search Client: chatbot in ...,Content Source: All Search Client: chatbot in ...,1.000000
164,Content Source: Customer Community Search Clie...,Content Source: Community Search Client: Custo...,0.949046
341,Content Source: Community Search Search Client...,Content Source: Search Unify Search Search Cli...,0.866055
155,Content Source: Solutions Search Client: CA\SY...,Content Source: Solutions Search Client: CA/SY...,0.863223
238,"Hi Team, We have received an P2 incident INC13...",Content Source: Wolken Search Client: CSM Cust...,0.807258
301,Content Source: Knowledge Base Search Client: ...,Content Source: Knowledge base Search Client: ...,0.802719
375,Content Source: Community Search Client: Compa...,Content Source: Compass Community & Support Po...,0.784277
375,Content Source: Community Search Client: Compa...,Content Source: Compass Community & Support Po...,0.773750
312,Content Source: Community Search Client: Compa...,Content Source: Community Search Client: Compa...,0.760983
411,Content Source: Community Search Client: Compa...,Content Source: Community Search Client: Compa...,0.756731


In [174]:
cosine_threshold_data = cosine_data.loc[(cosine_data['similarity_Score'] >= 0.5)]

In [175]:
df_groupby =  cosine_threshold_data.groupby(['Test_set_query','similarity_Score'])

In [176]:
len(df_groupby)

1625

In [177]:
#df_groupby.first().head(50)

In [178]:
data_new = df_groupby.first().reset_index()

In [179]:
data_new.head()

,Test_set_query,similarity_Score,Search_query
0,After a recent refresh of our NetSuite Sandbox...,0.506609,Content Source: Knowledge Base Search Client: ...
1,After a recent refresh of our NetSuite Sandbox...,0.515884,Content Source: Salesforce Knowledge Search Cl...
2,After a recent refresh of our NetSuite Sandbox...,0.524177,Content Source: Salesforce Search Client: Supp...
3,After a recent refresh of our NetSuite Sandbox...,0.564427,We are attempting to sync contacts between Net...
4,After a recent refresh of our NetSuite Sandbox...,0.581004,Content Source: Community Search Client: Commu...


In [180]:
df2.head(1)

,Status (Ticket),Ticket Id,Account Name,Created Time (Ticket),Ticket Closed Time,Subject,Agent Responded Time,Category (Ticket),Classifications,Due Date,First Response Time in Business Hours,Ticket Owner,Account Owner,Support Owner,RCA Category,Resolution,Ticket Description
0,Closed,2338.0,Kronos Incorporated,10/22/2020 06:46 PM,07/07/2022 05:54 PM,Search not returning expected results.,03/30/2021 07:20 PM,Search Client,Configurations,04/20/2021 06:46 PM,3816:33:00,Support Team,Neeraj Trikha,Omkar Dhiman,Backend Configurations,Client closed this case at their end. This was...,Search internally in production seems to not r...


In [181]:
data_join =  data_new.merge(df2, left_on= 'Search_query', right_on='Ticket Description', how='inner')[['Test_set_query','similarity_Score','Search_query','Ticket Id','Classifications']]

In [182]:
data_join

,Test_set_query,similarity_Score,Search_query,Ticket Id,Classifications
0,After a recent refresh of our NetSuite Sandbox...,0.506609,Content Source: Knowledge Base Search Client: ...,4516.0,Configurations
1,Content Source: Community Search Client: Compa...,0.504773,Content Source: Knowledge Base Search Client: ...,4516.0,Configurations
2,Content Source: Community Search Client: Compa...,0.503271,Content Source: Knowledge Base Search Client: ...,4516.0,Configurations
3,Content Source: Knowledge Base Search Client: ...,0.539627,Content Source: Knowledge Base Search Client: ...,4516.0,Configurations
4,Content Source: Salesforce KB Cases Search Cli...,0.636227,Content Source: Knowledge Base Search Client: ...,4516.0,Configurations
...,...,...,...,...,...
1627,"Hi Team, We have received an P2 incident INC13...",0.587872,Subject: I am unable to customize Wolken searc...,4018.0,Enhancement
1628,"Hi Team, We have received an P2 incident INC13...",0.603304,Content Source: bsnwolken-bst Search Client: I...,4739.0,Incident
1629,"We have a contact, Giacomo Corcione, that when...",0.509449,Adding Grazitti to the conversation as they ha...,4449.0,Incident
1630,"We have a contact, Giacomo Corcione, that when...",0.523953,"Hi Divanshi and Team, We are receiving reports...",4342.0,Incident


In [185]:
df1.head(2)

,Classifications,Category (Activity),Account Name,Ticket Id,CS Owner,Priority (Ticket),Created By (Ticket),Created Time (Ticket),Due Date,Ticket Age,...,Type 2,RCA Category,STAGE,Team,KB shared,Crawling,Happiness Rating,Support Feedback,Ticket Description,Resolution
1,-,-,Bluebeam,5020.0,Prashant Begra,Low,Support Admin,08/22/2022 09:21 PM,02/18/2023 10:21 PM,0.534479,...,-,-,-,-,-,False,-,-,Content Source: https://bl091912p.searchunify....,-
2,-,-,Broadcom,4967.0,Divanshi Arora,Low,Support Admin,08/10/2022 08:27 PM,02/06/2023 09:27 PM,12.571470,...,-,-,-,-,-,False,-,-,Content Source: BSN External Search Search Cli...,-


In [186]:
df2.head(1)

,Status (Ticket),Ticket Id,Account Name,Created Time (Ticket),Ticket Closed Time,Subject,Agent Responded Time,Category (Ticket),Classifications,Due Date,First Response Time in Business Hours,Ticket Owner,Account Owner,Support Owner,RCA Category,Resolution,Ticket Description
0,Closed,2338.0,Kronos Incorporated,10/22/2020 06:46 PM,07/07/2022 05:54 PM,Search not returning expected results.,03/30/2021 07:20 PM,Search Client,Configurations,04/20/2021 06:46 PM,3816:33:00,Support Team,Neeraj Trikha,Omkar Dhiman,Backend Configurations,Client closed this case at their end. This was...,Search internally in production seems to not r...


In [187]:
dat_new_ = data_join.groupby(['Test_set_query','similarity_Score'])

In [188]:
dat_new_.first().head(50)

Search_query  \
Test_set_query                                     similarity_Score                                                      
After a recent refresh of our NetSuite Sandbox ... 0.506609          Content Source: Knowledge Base Search Client: ...   
                                                   0.515884          Content Source: Salesforce Knowledge Search Cl...   
                                                   0.524177          Content Source: Salesforce Search Client: Supp...   
                                                   0.564427          We are attempting to sync contacts between Net...   
                                                   0.581004          Content Source: Community Search Client: Commu...   
                                                   0.667638          Please assist in this issue as soon as possibl...   
Based on our weekly calls, here’s what we’re lo... 0.517102          Content Source: Salesforce Search Client: Agen...   
                                                   0.550355          Content Source: Knowledge Base, Community, Aca...   
                                                   0.613711          Based on our weekly calls, here’s what we’re l...   
Content Source: A new JIRA instance Search Clie... 0.500338          Content Source: Core PAS Search Client: CyberA...   
                                                   0.501360          Content Source: Confluence Cloud Search Client...   
                                                   0.505434          Content Source: Salesforce Knowledge Search Cl...   
                                                   0.506365          Content Source: Bright Cove Search Client: Vaa...   
                                                   0.509309          Content Source: any Search Client: Support Por...   
                                                   0.509337          Content Source: Community Search Client: Commu...   
                                                   0.514550          Content Source: Community Search Client: Commu...   
                                                   0.516781          Content Source: Community, Documentation Searc...   
                                                   0.520480          Content Source: Solutions Search Client: CA/SY...   
                                                   0.537460          Content Source: All Search Client: Community S...   
                                                   0.537583          Content Source: Core PAS Search Client: CyberA...   
                                                   0.550298          Content Source: Salesforce Knowledge Search Cl...   
                                                   0.577316          Content Source: Jira Search Client: Service co...   
Content Source: All Search Client: Support Agen... 0.500374          Content Source: Community Search Client: Commu...   
                                                   0.504286          Content Source: Training (Docebo) Search Clien...   
                                                   0.505384          Content Source: All Search Client: Support Con...   
                                                   0.505718          Content Source: Community Search Client: Commu...   
                                                   0.507902          Content Source: CA/Sym Customer Support Portal...   
                                                   0.508700          Content Source: https://cyberark.searchunify.c...   
                                                   0.512989          Content Source: Multiple Search Client: API Pr...   
                                                   0.515357          Content Source: Caretend Search Client: Carete...   
                                                   0.516780          Content Source: All Search Client: All Product...   
                                                   0.518433          Content Sou

# CLASSIFICATION OF MISSING LABELS

In [189]:
dat_new_.first().to_excel('cosine_based_on_bert.xlsx')

In [190]:
%%capture
!pip install git+https://github.com/neuml/txtai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [191]:
%%capture

from txtai.pipeline import Labels

# # Create labels model
labels = Labels()

# Alternate models can be used via passing the model path as shown below
# labels = Labels("roberta-large-mnli")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [194]:
data = df1['Ticket Description'].tolist()

# List of labels
tags = ['Configurations', 'Enhancement', 'Incident', 'Bug', 'Question',
       'Others', 'Change Request']

print("%-75s %s" % ("Text", "Label"))
print("-" * 100)

for text in data:
    print("%-75s %s" % (text, tags[labels(text, tags)[0][0]]))

Text                                                                        Label
----------------------------------------------------------------------------------------------------
Content Source: https://bl091912p.searchunify.com/dashboard/content-sources/9 Search Client: ce069119-ef51-11e9-81f8-02ae30590d84 Product Version: Mamba 22 We were told that the "Design" tab in Search Client is an "on-demand" feature. Can we please get access to this team in our sandbox and production environments for all search clients. https://bl091910s.searchunify.com/ https://bl091912p.searchunify.com/ Thank you! Question
Content Source: BSN External Search Search Client: bsn external search client Product Version: Mamba 21 Do we have the specific BSN instance documentation used to create in CSP eCX Portal to BSN Connection Search Bar >> Hiya PROD https://portal.broadcom.com/group/support/brcmsearch Token: STG https://pstg-portal.broadcom.com/group/support/brcmsearch Token: Question
Content Source: htt

#ALTERNATIVE WAY

In [ ]:
#https://github.com/AYLIEN/fine-grained-event-classification/blob/dev/notebooks/SentenceTransformers-ZeroShot-Baseline.ipynb

##THIRD WAY

In [204]:
import os, gc, warnings
warnings.filterwarnings("ignore")

#import transformers
from transformers import pipeline

In [205]:
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [206]:
data = df1['Ticket Description'].tolist()

# List of labels
tags = ['Configurations', 'Enhancement', 'Incident', 'Bug', 'Question',
       'Others', 'Change Request']
classifier(data, tags, multi_label=True)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


[{'sequence': 'Content Source: https://bl091912p.searchunify.com/dashboard/content-sources/9 Search Client: ce069119-ef51-11e9-81f8-02ae30590d84 Product Version: Mamba 22 We were told that the "Design" tab in Search Client is an "on-demand" feature. Can we please get access to this team in our sandbox and production environments for all search clients. https://bl091910s.searchunify.com/ https://bl091912p.searchunify.com/ Thank you!',
  'labels': ['Question',
   'Change Request',
   'Bug',
   'Enhancement',
   'Configurations',
   'Others',
   'Incident'],
  'scores': [0.524315357208252,
   0.4307260513305664,
   0.33173102140426636,
   0.1979251652956009,
   0.07771528512239456,
   0.05670410394668579,
   0.026653600856661797]},
 {'sequence': 'Content Source: BSN External Search Search Client: bsn external search client Product Version: Mamba 21 Do we have the specific BSN instance documentation used to create in CSP eCX Portal to BSN Connection Search Bar >> Hiya PROD https://portal.b

# FEW SHOT CLASSIFICATION WITH SETFIT
#https://www.kaggle.com/code/nulldata/few-shot-text-classification-with-setfit

In [207]:
%%capture 
!pip install git+https://github.com/pmbaumgartner/setfit -q

In [208]:
from setfit import SetFitClassifier

In [209]:
# takes a sentence-transformers model
clf = SetFitClassifier("paraphrase-MiniLM-L3-v2")

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [214]:
data = df2['Ticket Description'].to_list()

# List of labels
tags = df2['Classifications'].to_list()


In [215]:
# fine-tunes embeddings + trains logistic regression head
clf.fit(data, tags) 

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/270 [00:00<?, ?it/s]

In [216]:
df1['predicted'] = clf.predict(df1['Ticket Description'].to_list())

In [220]:
df1[['Classifications','Ticket Description','predicted']]

,Classifications,Ticket Description,predicted
1,-,Content Source: https://bl091912p.searchunify....,Configurations
2,-,Content Source: BSN External Search Search Cli...,Configurations
3,-,Content Source: https://bc102102s.searchunify....,Configurations
4,-,Content Source: support.cohesity.com Search Cl...,Configurations
7,-,Content Source: n/a Search Client: Namely Prod...,Configurations
9,-,Content Source: Community Search Client: Searc...,Configurations
10,-,Content Source: Zoomin Search Client: Varonis ...,Bug
11,-,Content Source: All Search Client: Support Age...,Configurations
12,-,Content Source: Zoomin [product documentation]...,Question
14,Bug,Content Source: Solutions Search Client: CA\SY...,Configurations
